In [1]:
import pandas as pd

df = pd.read_excel("Skills.xlsx")
#print(df.head())

#Lets remove the importance values:

df2 = df.loc[df["Scale Name"] == "Level"]
df2.reset_index(drop = True, inplace = True)
#print(df2.head())

#Lets now remove the irrelevent columns:

df3 = df2.drop(columns = ["Scale ID","Scale Name","N","Recommend Suppress","Not Relevant","Date","Domain Source"])
print(df3.head())

#NOTE that we have ignored the suppress recomendations, we shall continue with this for now but will need to address this later


  O*NET-SOC Code             Title Element ID           Element Name  \
0     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension   
1     11-1011.00  Chief Executives    2.A.1.b       Active Listening   
2     11-1011.00  Chief Executives    2.A.1.c                Writing   
3     11-1011.00  Chief Executives    2.A.1.d               Speaking   
4     11-1011.00  Chief Executives    2.A.1.e            Mathematics   

   Data Value  Standard Error  Lower CI Bound  Upper CI Bound  
0        4.75            0.16            4.43            5.07  
1        4.88            0.23            4.43            5.32  
2        4.38            0.18            4.02            4.73  
3        4.88            0.13            4.63            5.12  
4        3.62            0.26            3.11            4.14  


In [2]:
#Lets now discover a bit about our data set:

df3.info()

#We note that there are some occupations for which the standard error and bound values are missing. Lets supress these for now:

df3.drop(columns = ["Standard Error","Lower CI Bound","Upper CI Bound"],inplace = True)
print(df3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30555 entries, 0 to 30554
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   O*NET-SOC Code  30555 non-null  object 
 1   Title           30555 non-null  object 
 2   Element ID      30555 non-null  object 
 3   Element Name    30555 non-null  object 
 4   Data Value      30555 non-null  float64
 5   Standard Error  28700 non-null  float64
 6   Lower CI Bound  28700 non-null  float64
 7   Upper CI Bound  28700 non-null  float64
dtypes: float64(4), object(4)
memory usage: 1.9+ MB
  O*NET-SOC Code             Title Element ID           Element Name  \
0     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension   
1     11-1011.00  Chief Executives    2.A.1.b       Active Listening   
2     11-1011.00  Chief Executives    2.A.1.c                Writing   
3     11-1011.00  Chief Executives    2.A.1.d               Speaking   
4     11-1011.00  Chief Execut

In [ ]:
#We now want to create a new dataframe that has occupation titles as observations, and the different skills as variables

#We need to begin by creating a dictionary:

mydict = {}
variable = df3.columns[0]
myset = set((df3[variable]))
variable_list = list(myset)
#print(variable_list)
mydict[variable] = variable_list

variable = df3.columns[1]
myset = set((df3[variable]))
variable_list = list(myset)
#print(variable_list)
mydict[variable] = variable_list

df4 = pd.DataFrame(mydict)
print(df4.head())

#Evidently this code doesn't work the way I want it to

In [33]:
#I shall implement the drop_duplicates method:

df4 = df3[["O*NET-SOC Code","Title"]]
df4.drop_duplicates(inplace=True)
df4.reset_index(drop = True, inplace = True)
#print(df4.head())

#We now need to add the variables. Begin by adding empty columns to the dataframe:

n_jobs = len(set((df3["Title"])))
n_variables = len(set((df3["Element Name"])))

for i in range(n_jobs):
    df4[df3["Element Name"][i]] = ""

#print(df4.head())
#Now we need to fill these columns:

x = df3.loc[df3["Title"] == "Chief Executives"]
y = x["Data Value"]

for i in range(n_variables):
    df4[df4.columns[2+i]][0] = y[i]
    
#We now need to do this procedure for every job:

for j in range(n_jobs):
    x = df3.loc[df3["Title"] == df4.iloc[j,1]]
    y = x["Data Value"]
    y.reset_index(drop = True, inplace = True)
    for i in range(n_variables):
        df4[df4.columns[2+i]][j] = y[i]


<ipython-input-33-398803c0bc0e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop_duplicates(inplace=True)
<ipython-input-33-398803c0bc0e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4[df3["Element Name"][i]] = ""
C:\Users\jacob\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user

In [35]:
print(df4.tail())

    O*NET-SOC Code                                             Title  \
868     53-7071.00  Gas Compressor and Gas Pumping Station Operators   
869     53-7072.00           Pump Operators, Except Wellhead Pumpers   
870     53-7073.00                                  Wellhead Pumpers   
871     53-7081.00         Refuse and Recyclable Material Collectors   
872     53-7121.00                 Tank Car, Truck, and Ship Loaders   

    Reading Comprehension Active Listening Writing Speaking Mathematics  \
868                  3.12              3.0    2.88      3.0        2.12   
869                   3.0              3.0    2.75     2.88        2.62   
870                  2.88             2.75    2.12      3.0        1.62   
871                  2.38             2.62    2.38     2.25         0.5   
872                  3.12              3.0    2.75     2.88        2.12   

    Science Critical Thinking Active Learning  ... Troubleshooting Repairing  \
868    1.25               3.0       

### We now have the skills dataframe just as we want it!

In [ ]:
#Lets load in 